## Langchain  - memory

In [2]:

import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory


In [3]:
# Load your OpenAI API key from a .env file
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")


In [4]:
# Initialize the language model (using gpt-4o-mini)
model = ChatOpenAI(model="gpt-4o-mini", openai_api_key=openai_api_key)

* Conversational Memory

This function creates in-memory chat history (new chat history) for each unique session ID or returns an existing one.

In [5]:
# Create an in-memory chat history store
store = {}

def get_session_history(session_id: str):
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]


get_session_history() retrieves the chat history that belongs to a particular session (e.g., a user's conversation). If the session does not already have a history stored, it creates a new one.

#### Different type message history implementation
**In-memory implementations
🗄️InMemoryChatMessageHistory**
from langchain_core.chat_history import InMemoryChatMessageHistory
- Stores messages in Python memory
- Lost when the program stops
- Fast, no external dependencies
- Best for prototyping


**💾 File-based implementations
 FileChatMessageHistory**
 
from langchain_core.chat_history import FileChatMessageHistory

- Stores messages in a local file (usually JSON)
- Persisted across runs
- Good for single-user apps or desktop tools

**🛢️ Database-backed implementations
 SQLChatMessageHistory**
 
from langchain_core.chat_history import SQLChatMessageHistory
- Saves messages in SQLite / Postgres
- Suitable for production apps
- Supports multiple users and scaling

**RedisChatMessageHistory**

from langchain_core.chat_history import RedisChatMessageHistory
- Stores chat history in Redis
- Allows TTL (auto-expire messages)
- Best for web-scale and session-based chat

**☁️ Cloud-native / external store

 MongoDBChatMessageHistory**
- Stores messages in MongoDB
- Flexible document-based structure

**DynamoDBChatMessageHistory
AWS DynamoDB storage**
- Fully managed serverless persistence

**🧭 Specialized histories
 BullMQChatMessageHistory**
 
- For Node-based queue-backed workflows

**ChatMessageHistoryWithSummary**
- Wraps another history store
- Adds summarization to prevent context overflow

In [6]:

# Create a prompt template for the chatbot
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Answer all questions to the best of your ability."),
    MessagesPlaceholder(variable_name="messages")
])



In [7]:

# Conversation with history handling
session_id = "abc123"  # Unique session identifier


In [8]:

# Combine the model with the prompt and message history
chain = prompt | model

with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

# RunnableWithMessageHistory is a LangChain wrapper that adds chat history capability to any runnable chain.
# RunnableWithMessageHistory is agnostic as to how the get_session_history callable retrieves its chat message histories.

In [9]:

# Start a conversation
print("Chatbot: Hi! How can I assist you today?")

while True:
    user_input = input("You: ")

    # Exit condition
    if user_input.lower() in ["exit", "quit"]:
        print("Chatbot: Goodbye!")
        break

    # Stream response from the model
    print("Chatbot: ", end="", flush=True)
    for r in with_message_history.stream(
            {"messages": [HumanMessage(content=user_input)]},
            config={"configurable": {"session_id": session_id}}
    ):
        print(r.content, end="", flush=True)
    print()  #


Chatbot: Hi! How can I assist you today?


You:  Can you tell me when MS Dhoni will retire?


Chatbot: As of my last update in October 2023, MS Dhoni has not publicly announced a specific date for his retirement from cricket. He retired from international cricket on August 15, 2020, but continues to play in franchise cricket, including the Indian Premier League (IPL) for the Chennai Super Kings. Any news regarding his future retirement would need to be checked from the latest sources or updates.


You:  I mean from IPL


Chatbot: As of my last knowledge update in October 2023, MS Dhoni had not officially announced a retirement date from the Indian Premier League (IPL). He continued to play for the Chennai Super Kings in the IPL. For the latest updates on his retirement or participation in future seasons, it's best to check recent news sources or official announcements from the IPL or Chennai Super Kings.


You:  exit


Chatbot: Goodbye!
